In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
subm = pd.read_csv('../input/sample_submission.csv')

In [ ]:
subm.head()

In [ ]:
train.head()

In [ ]:
train['comment_text'][0]

In [ ]:
train['comment_text'][4]

In [ ]:
lens = train.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

In [ ]:
lens.hist();

In [ ]:
train_new = train[['id','target','comment_text']]

In [ ]:
train_new.head()

In [ ]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
 train_new.shape[0]

In [ ]:
train_new.target = train_new.target.apply(lambda x: 0 if x <=0.5 else 1).copy()

In [ ]:
train1=pd.get_dummies(train_new.target)

In [ ]:
train_new=train_new.drop('target',axis=1)

In [ ]:
train_new.head()

In [ ]:
train_data = pd.concat([train_new,train1],axis=1)

In [ ]:
train_data.head()

In [ ]:
train_data.columns = ['id','comment_text','Traget_zero','Target_One']

In [ ]:
train_data.head()

In [ ]:
n = train_data.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train_data['comment_text'])
test_term_doc = vec.transform(test['comment_text'])

In [ ]:
trn_term_doc, test_term_doc

In [ ]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [ ]:
x = trn_term_doc
test_x = test_term_doc

In [ ]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=0.1, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [ ]:
label_cols = ['Traget_zero','Target_One']

In [ ]:
preds = np.zeros((len(test), len(label_cols)))

In [ ]:
for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train_data[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

In [ ]:
submid = pd.DataFrame({'id': subm["id"]})
submission1 = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)


In [ ]:
submission1.head()

In [ ]:
submission1=submission1.drop(columns=['Traget_zero'])

In [ ]:
submission1.columns=['id','prediction']

In [ ]:
submission1.head()

In [ ]:
submission1.to_csv('submission.csv', index=False)

In [ ]:
xx=pd.read_csv('../working/submission.csv')

In [ ]:
xx.head()